In [1]:
from data_gen_utils import *
import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
import matplotlib.pyplot as plt
import mpmath

from synthetic.generate import SingleTaskTreeDepsGenerator
from metal.label_model import LabelModel, CliqueTree
from metal.label_model.utils import (
    compute_mu,
    compute_covariance,
    compute_inv_covariance,
    print_matrix,
    visualize_matrix
)

/lfs/1/annhe/anaconda3/envs/metal/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
M = 6
n = 10000
K = 1
dg = DataGenerator(M,n)

In [3]:
dg.generate_O(k=3)

In [4]:
print(dg.deps)

[(0, 1), (2, 3), (4, 5)]


In [5]:
# remove deps
print(dg.deps_no_diag)

[(0, 1), (2, 3), (4, 5)]


In [6]:
print(dg.L)

w,h = (dg.L).shape
L = np.zeros((w,h),dtype=float)
for i in range(w):
    for j in range(h):
        if dg.L[i,j] > 0:
            L[i,j] = 1
        else:
            L[i,j] = 0

[[ 1. -1.  1. ...  1.  1. -1.]
 [ 1. -1.  1. ... -1. -1. -1.]
 [-1. -1.  1. ...  1. -1. -1.]
 [-1. -1.  1. ...  1. -1. -1.]
 [-1.  1.  1. ... -1. -1.  1.]
 [-1. -1.  1. ... -1. -1. -1.]]


In [7]:
print(L)
print(dg.deps_no_diag)

[[1. 0. 1. ... 1. 1. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 1. 1. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 0.]]
[(0, 1), (2, 3), (4, 5)]


In [8]:
lm = LabelModel(k=1)
lm.config['higher_order_cliques'] = False
lm.config['verbose'] = True

In [9]:
lm.train(L=dg.L.T,
         deps=dg.deps_no_diag,
         all_unary_cliques=True,
        higher_order_cliques=True,
        n_epochs=50000,
        print_every=5000,
        lr=0.0001,
        l2=0,
        O_inv_prec=1024,)

I GOT HERE
I GOT HERE 2
I GOT HERE 3
INIT 1
GET CLIQUE TREE 1
GET CLIQUE TREE 2
GET CLIQUE TREE 3
GET CLIQUE TREE 4
GET CLIQUE TREE 5
INIT 2
CLIQUE TREE 1
START INDEX:  9
I GOT HERE 4
L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
(9, 9)
[[0.5104 0.2853 0.349  0.2595 0.2803 0.2102 0.1721 0.2853 0.2472]
 [0.2853 0.3143 0.2701 0.2132 0.1862 0.169  0.137  0.2853 0.2027]
 [0.349  0.2701 0.5044 0.3273 0.2936 0.2435 0.1989 0.245  0.3273]
 [0.2595 0.2132 0.3273 0.3439 0.2101 0.1932 0.1578 0.1934 0.3273]
 [0.2803 0.1862 0.2936 0.2101 0.5003 0.2289 0.2289 0.1688 0.2   ]
 [0.2102 0.169  0.2435 0.1932 0.2289 0.2799 0.2289 0.1531 0.1836]
 [0.1721 0.137  0.1989 0.1578 0.2289 0.2289 0.2289 0.1241 0.1499]
 [0.2853 0.2853 0.245  0.1934 0.1688 0.1531 0.1241 0.2853 0.1838]
 [0.2472 0.2027 0.3273 0.3273 0.2    0.1836 0.1499 0.1838 0.3273]]
Computing O^{-1}...
L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
O unnorm   [5104.0  2853.0  3490.0  2595.0  2803.0  2102.0  1721.0

In [10]:
lm._set_constants(dg.L.T)
#lm._set_dependencies(dg.deps_no_diag)
mu = compute_mu(lm._get_augmented_label_matrix(dg.L.T), dg.Y.T, K, np.full(K, 1/K))


# Test against the true parameter values
mu_est = lm.mu.detach().numpy()
print(mu_est)
print(f"Average absolute error: {np.mean(np.abs(mu_est - mu))}")

mu_est_sm = mu_est[:6]
mu_true = dg.mu
print(f"Average absolute error: {np.mean(np.abs(mu_est_sm - mu_true))}")

L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
[[0.5612317 ]
 [0.41689733]
 [0.6085657 ]
 [0.45181268]
 [0.48982117]
 [0.3751694 ]
 [0.30683342]
 [0.37859803]
 [0.43042484]]
Average absolute error: 0.19251990027954127
Average absolute error: 0.16827771406173708


In [11]:
lm.train(L=L.T,
         deps=dg.deps_no_diag,
         all_unary_cliques=True,
        higher_order_cliques=True,
        n_epochs=50000,
        print_every=5000,
        lr=0.0001,
        l2=0,
        O_inv_prec=1024,)

I GOT HERE
I GOT HERE 2
I GOT HERE 3
INIT 1
GET CLIQUE TREE 1
GET CLIQUE TREE 2
GET CLIQUE TREE 3
GET CLIQUE TREE 4
GET CLIQUE TREE 5
INIT 2
CLIQUE TREE 1
START INDEX:  9
I GOT HERE 4
L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
Using cached O_inv...
(9, 9)
[[0.5104 0.2853 0.349  0.2595 0.2803 0.2102 0.1721 0.2853 0.2472]
 [0.2853 0.3143 0.2701 0.2132 0.1862 0.169  0.137  0.2853 0.2027]
 [0.349  0.2701 0.5044 0.3273 0.2936 0.2435 0.1989 0.245  0.3273]
 [0.2595 0.2132 0.3273 0.3439 0.2101 0.1932 0.1578 0.1934 0.3273]
 [0.2803 0.1862 0.2936 0.2101 0.5003 0.2289 0.2289 0.1688 0.2   ]
 [0.2102 0.169  0.2435 0.1932 0.2289 0.2799 0.2289 0.1531 0.1836]
 [0.1721 0.137  0.1989 0.1578 0.2289 0.2289 0.2289 0.1241 0.1499]
 [0.2853 0.2853 0.245  0.1934 0.1688 0.1531 0.1241 0.2853 0.1838]
 [0.2472 0.2027 0.3273 0.3273 0.2    0.1836 0.1499 0.1838 0.3273]]
Estimating Z...
[Epoch 0] Loss: 195.888931
[Epoch 1000] Loss: 5.775929
[Epoch 2000] Loss: 5.775927
[Epoch 3000] Loss: 5.775927
[

In [12]:
lm._set_constants(L.T)
#lm._set_dependencies(dg.deps_no_diag)
Y = np.zeros(dg.Y.shape,dtype=float)
for i in range(n):
    if dg.Y[i] > 0:
        Y[i] = 1
    else:
        Y[i] = 0
mu = compute_mu(lm._get_augmented_label_matrix(L.T), Y.T, K, np.full(K, 1/K))


# Test against the true parameter values
mu_est = lm.mu.detach().numpy()
print(mu_est)
print(f"Average absolute error: {np.mean(np.abs(mu_est - mu))}")

mu_est_sm = mu_est[:6]
mu_true = dg.mu
print(f"Average absolute error: {np.mean(np.abs(mu_est_sm - mu_true))}")

L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
[[0.5612265 ]
 [0.41689497]
 [0.60856587]
 [0.4518128 ]
 [0.4898216 ]
 [0.37516692]
 [0.3068334 ]
 [0.37860107]
 [0.430425  ]]
Average absolute error: 0.1925205791102179
Average absolute error: 0.16827818758752613


### Changing learning rate

In [13]:
lm.train(L=L.T,
         deps=dg.deps_no_diag,
         all_unary_cliques=True,
        higher_order_cliques=True,
        n_epochs=100000,
        print_every=5000,
        lr=0.0001,
        l2=0,
        O_inv_prec=1024,)

I GOT HERE
I GOT HERE 2
I GOT HERE 3
INIT 1
GET CLIQUE TREE 1
GET CLIQUE TREE 2
GET CLIQUE TREE 3
GET CLIQUE TREE 4
GET CLIQUE TREE 5
INIT 2
CLIQUE TREE 1
START INDEX:  9
I GOT HERE 4
L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
Using cached O_inv...
(9, 9)
[[0.5104 0.2853 0.349  0.2595 0.2803 0.2102 0.1721 0.2853 0.2472]
 [0.2853 0.3143 0.2701 0.2132 0.1862 0.169  0.137  0.2853 0.2027]
 [0.349  0.2701 0.5044 0.3273 0.2936 0.2435 0.1989 0.245  0.3273]
 [0.2595 0.2132 0.3273 0.3439 0.2101 0.1932 0.1578 0.1934 0.3273]
 [0.2803 0.1862 0.2936 0.2101 0.5003 0.2289 0.2289 0.1688 0.2   ]
 [0.2102 0.169  0.2435 0.1932 0.2289 0.2799 0.2289 0.1531 0.1836]
 [0.1721 0.137  0.1989 0.1578 0.2289 0.2289 0.2289 0.1241 0.1499]
 [0.2853 0.2853 0.245  0.1934 0.1688 0.1531 0.1241 0.2853 0.1838]
 [0.2472 0.2027 0.3273 0.3273 0.2    0.1836 0.1499 0.1838 0.3273]]
Estimating Z...
[Epoch 0] Loss: 257.923523
[Epoch 1000] Loss: 5.775932
[Epoch 2000] Loss: 5.775927
[Epoch 3000] Loss: 5.775927
[

In [14]:
lm._set_constants(L.T)
#lm._set_dependencies(dg.deps_no_diag)
Y = np.zeros(dg.Y.shape,dtype=float)
for i in range(n):
    if dg.Y[i] > 0:
        Y[i] = 1
    else:
        Y[i] = 0
mu = compute_mu(lm._get_augmented_label_matrix(L.T), Y.T, K, np.full(K, 1/K))


# Test against the true parameter values
mu_est = lm.mu.detach().numpy()
print(mu_est)
print(f"Average absolute error: {np.mean(np.abs(mu_est - mu))}")

mu_est_sm = mu_est[:6]
mu_true = dg.mu
print(f"Average absolute error: {np.mean(np.abs(mu_est_sm - mu_true))}")

L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
[[0.56123185]
 [0.41689482]
 [0.6085658 ]
 [0.45181024]
 [0.48982129]
 [0.3751668 ]
 [0.30683324]
 [0.37859824]
 [0.43042746]]
Average absolute error: 0.1925203969849144
Average absolute error: 0.16827853859265648


In [15]:
from amc_utils import *
O = dg.O
O_inv = dg.Oinv
real_deps = dg.deps
mu_rec = solveMatrixCompletionWithMu(O_inv,O,real_deps)
print(f"Average absolute error: {np.mean(np.abs(mu_rec - mu_true))}")

Average absolute error: 0.08422952498613263
